In [1]:
import torch
import torch.nn as nn
import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0, "/mnt/disk1/nguyen.thanh.huyenb/yolov7_train/")

/home/nguyen.thanh.huyenb/miniconda3/envs/mmdetection_v2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
from models.experimental import attempt_load
weights = "/mnt/disk1/nguyen.thanh.huyenb/yolov7_train/YOLOv7/base_weights/epoch_002.pt"
model = attempt_load(weights, map_location=device) 
model.eval()

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


Model(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (3): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (4): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): SiLU(inplace=True)
    )
    (5): Conv(
      (conv): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): SiLU(inplace=True)
    )
    (6): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (7): Conv(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), strid

In [4]:
stride = int(model.stride.max()) 
stride
names = model.module.names if hasattr(model, 'module') else model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]
names, colors

(['polyp'], [[73, 59, 18]])

In [5]:
model.gr

1.0

## Load image

In [14]:
from utils.datasets import letterbox
from utils.general import scale_coords, non_max_suppression
from utils.semi_psuedo_label_process import non_max_suppression_pseudo_decouple, non_max_suppression_pseudo_decouple_multi_view, xyxy2xywhn


In [15]:
# 3_244, 160,
paths = ['/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/12303.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/22037.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/16006.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/13021.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/22438.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/21530.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/5966.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/11259.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/19036.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/14941.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/14393.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/2714.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/8063.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/2500.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/28623.jpg', '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/1564.jpg']

In [16]:
image_path = "/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/valid/images/1_161.jpg"
# image_path = path[3]
# image_path = "/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/1564.jpg"
# image_path = paths[-1]
# image_path = "/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/21210.jpg"
image_path = "/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/11141.jpg" # too small
print(image_path)
image0 = cv2.imread(image_path)
image0.shape
image0

/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/11141.jpg


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [17]:
image = letterbox(image0, (640, 640), stride=stride)[0]
image = image[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
image = np.ascontiguousarray(image)
image.shape

(3, 480, 640)

In [18]:
image = torch.from_numpy(image).to(device).float()
image /= 255.0
image = image.unsqueeze(0)
image

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')

In [19]:
with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
    out = model(image)
with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
    out_augment = model(image, augment=True)
out[0].shape, out_augment[0].shape

(torch.Size([1, 18900, 6]), torch.Size([1, 42525, 6]))

In [20]:

def draw_image(out, image, image0, filename="out.png"):
    pred = non_max_suppression(out[0], conf_thres=0.01)
    print(pred)
    det = pred[0]
    det[:, :4] = scale_coords(image.shape[2:], det[:, :4], image0.shape).round()
   
    for *xyxy, conf, cls in reversed(det):
        # Convert xyxy format to xywh format
        xyxy = [int(i.cpu()) for i in xyxy]
        x, y, w, h =  xyxy[0], xyxy[1], xyxy[2]-xyxy[0], xyxy[3]-xyxy[1]

        # Draw the bounding box on the image
        image0 = cv2.rectangle(image0, (x, y), (x+w, y+h), (0, 255, 0), 1)
        text = f"{names[int(cls)]}: {conf:.2f}"
        image0 = cv2.putText(image0, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.25, (0, 255, 0), 1)
        label = f'{names[int(cls)]} {conf:.2f}'
        print(label)
    cv2.imwrite(filename, image0)
   

draw_image(out, image, image0, "out.png")
draw_image(out_augment, image, image0, "out_aug.png")

[tensor([[2.55444e+02, 2.70138e+02, 4.72945e+02, 4.70218e+02, 4.65297e-02, 0.00000e+00],
        [1.53655e+02, 7.92975e+01, 2.58119e+02, 1.68929e+02, 4.07364e-02, 0.00000e+00],
        [1.80677e+02, 2.47152e+02, 2.77451e+02, 3.44350e+02, 2.82048e-02, 0.00000e+00],
        [2.53760e+02, 1.65212e+02, 4.18308e+02, 3.23871e+02, 2.57655e-02, 0.00000e+00],
        [3.02135e+02, 1.47237e+02, 4.34472e+02, 2.55250e+02, 2.38604e-02, 0.00000e+00],
        [2.73790e+02, 1.20666e+02, 4.10652e+02, 2.28049e+02, 1.78420e-02, 0.00000e+00],
        [2.90480e+02, 2.52042e+02, 4.48096e+02, 3.75615e+02, 1.05000e-02, 0.00000e+00]], device='cuda:0')]
polyp 0.01
polyp 0.02
polyp 0.02
polyp 0.03
polyp 0.03
polyp 0.04
polyp 0.05
[tensor([[2.55444e+02, 2.70138e+02, 4.72945e+02, 4.70218e+02, 4.65297e-02, 0.00000e+00],
        [1.53655e+02, 7.92975e+01, 2.58119e+02, 1.68929e+02, 4.07364e-02, 0.00000e+00],
        [3.83589e+02, 1.41466e+02, 5.40081e+02, 2.81938e+02, 3.72221e-02, 0.00000e+00],
        [2.71829e+02, 

## Dataloader

In [109]:
from utils.datasets import create_dataloader
from utils.datasets import LoadImagesAndLabels
from utils.datasets_semi import LoadImagesAndLabelsSemi
from utils.plots import plot_images
import yaml
from utils.general import check_img_size
import os

In [216]:
path = "/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/Semi_PolypsSet/semi_1_labels_fold_2_percent_10/train2019.2@10-unlabeled.json"
img_size = [640,640]
rect = False
batch_size = 4
augment=True
cache=False
pad=0.0
rect=False
stride=32             
rank=-1
world_size=1
workers=8
image_weights=False
quad=False
prefix=''
gs = stride = 32
imgsz,imgsz_test = [check_img_size(x, gs) for x in img_size]
imgsz,imgsz_test

(640, 640)

In [217]:
def load_hyperparameter(hyp_path):
        """Hyperparameters"""
        with open(hyp_path) as f:
            hyp = yaml.load(f, Loader=yaml.SafeLoader)
        return hyp
hyp = load_hyperparameter("/mnt/disk1/nguyen.thanh.huyenb/yolov7_train/data/hyp.scratch.p5.yaml")
hyp["mosaic"] = 0

In [218]:
dataset = LoadImagesAndLabelsSemi(path, imgsz, batch_size,
                                      augment=True,  # augment images
                                      hyp=hyp,  # augmentation hyperparameters
                                      rect=rect,  # rectangular training
                                      cache_images=cache,
                                      single_cls=True,
                                      stride=int(stride),
                                      pad=pad,
                                      mosaic=True,
                                      image_weights=image_weights,
                                      prefix=prefix)

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/labels/1.txt
/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/Semi_PolypsSet/semi_1_labels_fold_2_percent_10/train2019.2@10-unlabeled.json
/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/Semi_PolypsSet/semi_1_labels_fold_2_percent_10/train2019.2@10-unlabeled.cache


Scanning '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/Semi_PolypsSet/semi_1_labels_fold_2_percent_10/train2019.2@10-unlabeled.cache' images and labels... 26015 found, 0 missing, 1540 empty, 0 corrupted: 100%|██████████| 26015/26015 [00:00<?, ?it/s]


In [219]:
from utils.datasets import InfiniteDataLoader

batch_size = min(batch_size, len(dataset))
nw = min([os.cpu_count() // world_size, batch_size if batch_size > 1 else 0, workers])  # number of workers
sampler = torch.utils.data.distributed.DistributedSampler(dataset) if rank != -1 else None
loader = torch.utils.data.DataLoader if image_weights else InfiniteDataLoader
# Use torch.utils.data.DataLoader() if dataset.properties will update during training else InfiniteDataLoader()
dataloader = loader(dataset,
                    batch_size=batch_size,
                    num_workers=nw,
                    sampler=sampler,
                    pin_memory=True,
                    shuffle=True,
                    collate_fn=LoadImagesAndLabelsSemi.collate_fn4 if quad else LoadImagesAndLabelsSemi.collate_fn)

In [220]:
# img, labels_out, path, shapes =  next(iter(dataloader))
label_imgs, label_targets, path, shape =  next(iter(dataloader))
(img, img_strong) = label_imgs 
labels_out = label_targets
plot_images(img, labels_out, path, "images.jpg")
labels_out, path

(tensor([[0.00000, 0.00000, 0.55382, 0.40147, 0.14958, 0.14842],
         [1.00000, 0.00000, 0.42410, 0.59895, 0.17713, 0.18930],
         [2.00000, 0.00000, 0.47481, 0.58270, 0.04930, 0.04930],
         [3.00000, 0.00000, 0.51383, 0.26174, 0.40279, 0.50021]]),
 ['/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/16741.jpg',
  '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/3495.jpg',
  '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/633.jpg',
  '/mnt/disk1/nguyen.thanh.huyenb/PolypsSet/PolypsSet/train2019/images/9520.jpg'])

In [221]:
img = img.float()
img /= 255
img

tensor([[[[0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          ...,
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706]],

         [[0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          ...,
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706]],

         [[0.44706, 0.44706, 0.44706,  ..., 0.44706, 0.44706, 0.44706],
          [0.44706, 0.44706, 0

In [222]:
with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
    out = model(img.to(device))
with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
    out_aug = model(img.to(device), augment=True)
out[0].shape, out_aug[0].shape

(torch.Size([4, 25200, 6]), torch.Size([4, 55755, 6]))

In [223]:
conf_thres = 0.05
cls_thres = 0.1
iod_thres = 0.5

In [224]:
pred = non_max_suppression(out[0][:, : , :] ,conf_thres=conf_thres, iou_thres=0.5)
pred

[tensor([[2.99351e+02, 1.90204e+02, 4.22218e+02, 2.97807e+02, 6.22224e-01, 0.00000e+00],
         [1.33132e+02, 1.37697e+02, 3.39819e+02, 3.53421e+02, 1.21538e-01, 0.00000e+00],
         [2.63893e+02, 5.36065e+01, 4.75085e+02, 2.98887e+02, 1.09125e-01, 0.00000e+00]], device='cuda:0'),
 tensor([[2.89632e+02, 2.83420e+02, 5.73448e+02, 5.79217e+02, 5.91744e-02, 0.00000e+00]], device='cuda:0'),
 tensor([[2.33090e+02, 2.19715e+02, 5.13446e+02, 4.59726e+02, 1.08198e-01, 0.00000e+00]], device='cuda:0'),
 tensor([[1.96078e+02, 1.28332e+02, 4.00658e+02, 3.35670e+02, 3.07579e-01, 0.00000e+00],
         [4.31967e+02, 1.52553e+02, 5.66011e+02, 2.70786e+02, 6.68286e-02, 0.00000e+00]], device='cuda:0')]

In [225]:
pred1, pred_cls = non_max_suppression_pseudo_decouple(out[0],conf_thres=conf_thres, iou_thres=0.45)
pred1, pred_cls

([tensor([[2.99351e+02, 1.90204e+02, 4.22218e+02, 2.97807e+02, 6.12622e-01, 0.00000e+00],
          [1.33132e+02, 1.37697e+02, 3.39819e+02, 3.53421e+02, 1.19500e-01, 0.00000e+00],
          [2.63893e+02, 5.36065e+01, 4.75085e+02, 2.98887e+02, 1.07534e-01, 0.00000e+00]], device='cuda:0'),
  tensor([[2.89632e+02, 2.83420e+02, 5.73448e+02, 5.79217e+02, 5.82581e-02, 0.00000e+00]], device='cuda:0'),
  tensor([[2.33090e+02, 2.19715e+02, 5.13446e+02, 4.59726e+02, 1.06779e-01, 0.00000e+00]], device='cuda:0'),
  tensor([[1.96078e+02, 1.28332e+02, 4.00658e+02, 3.35670e+02, 3.03018e-01, 0.00000e+00],
          [4.31967e+02, 1.52553e+02, 5.66011e+02, 2.70786e+02, 6.59390e-02, 0.00000e+00]], device='cuda:0')],
 [tensor([[299.35110, 190.20398, 422.21829, 297.80676,   0.61262,   0.00000]], device='cuda:0'),
  tensor([], device='cuda:0', size=(0, 6)),
  tensor([], device='cuda:0', size=(0, 6)),
  tensor([[1.96078e+02, 1.28332e+02, 4.00658e+02, 3.35670e+02, 3.03018e-01, 0.00000e+00]], device='cuda:0')]

In [226]:
pred1_aug, _ = non_max_suppression_pseudo_decouple(out_aug[0],conf_thres=conf_thres, iou_thres=0.45)
pred1_aug

[tensor([[2.99351e+02, 1.90204e+02, 4.22218e+02, 2.97807e+02, 6.12622e-01, 0.00000e+00],
         [1.33132e+02, 1.37697e+02, 3.39819e+02, 3.53421e+02, 1.19500e-01, 0.00000e+00],
         [2.63893e+02, 5.36065e+01, 4.75085e+02, 2.98887e+02, 1.07534e-01, 0.00000e+00],
         [2.87311e+02, 1.55445e+02, 4.97324e+02, 3.57888e+02, 5.60933e-02, 0.00000e+00]], device='cuda:0'),
 tensor([[2.89632e+02, 2.83420e+02, 5.73448e+02, 5.79217e+02, 5.82581e-02, 0.00000e+00],
         [2.23748e+02, 3.91463e+02, 3.91374e+02, 5.63763e+02, 5.71005e-02, 0.00000e+00],
         [1.41882e+02, 3.00451e+02, 4.08325e+02, 5.85907e+02, 5.40932e-02, 0.00000e+00]], device='cuda:0'),
 tensor([[2.33090e+02, 2.19715e+02, 5.13446e+02, 4.59726e+02, 1.06779e-01, 0.00000e+00]], device='cuda:0'),
 tensor([[1.96078e+02, 1.28332e+02, 4.00658e+02, 3.35670e+02, 3.03018e-01, 0.00000e+00],
         [4.31967e+02, 1.52553e+02, 5.66011e+02, 2.70786e+02, 6.59390e-02, 0.00000e+00]], device='cuda:0')]

In [227]:
from utils.general import xyxy2xywh
from copy import deepcopy
def convert_output_to_label(imgs, preds, shapes):
    labels = torch.zeros(0, 7).to(device)
   
    for i, pred in enumerate(preds):
        label = torch.zeros(len(pred), 7).to(device)
        h0, w0  = shapes[i][0]
        box = scale_coords(imgs[i].shape[:2], pred[:, 0:4], shape[i][0], shape[i][1]).round()
        box = xyxy2xywhn(pred[:, 0:4], w0, h0)
        label[:, 0] = i
        label[:, 1] = pred[:, 5]
        label[:, 2:6] = box
        label[:, 6] = pred[:, 4]
        labels = torch.cat([labels, label])
    
    return labels

pseudo_label = convert_output_to_label(img, pred, shape)
pseudo_label

tensor([[0.00000, 0.00000, 0.56443, 0.34831, 0.19222, 0.21478, 0.62222],
        [0.00000, 0.00000, 0.36995, 0.35142, 0.32335, 0.43059, 0.12154],
        [0.00000, 0.00000, 0.57804, 0.22893, 0.33040, 0.45786, 0.10912],
        [1.00000, 0.00000, 0.67428, 0.71190, 0.44346, 0.57621, 0.05917],
        [2.00000, 0.00000, 0.58323, 0.54108, 0.43806, 0.50002, 0.10820],
        [3.00000, 0.00000, 0.46620, 0.31667, 0.31966, 0.43196, 0.30758],
        [3.00000, 0.00000, 0.77967, 0.27431, 0.20944, 0.24632, 0.06683]], device='cuda:0')

In [228]:
pseudo_label1 = convert_output_to_label(img, pred1, shape)
pseudo_label1_aug = convert_output_to_label(img, pred1_aug, shape)
pseudo_label1_cls = convert_output_to_label(img, pred_cls, shape)
pseudo_label1, pseudo_label1_aug

(tensor([[0.00000, 0.00000, 0.56443, 0.34831, 0.19222, 0.21478, 0.61262],
         [0.00000, 0.00000, 0.36995, 0.35142, 0.32335, 0.43059, 0.11950],
         [0.00000, 0.00000, 0.57804, 0.22893, 0.33040, 0.45786, 0.10753],
         [1.00000, 0.00000, 0.67428, 0.71190, 0.44346, 0.57621, 0.05826],
         [2.00000, 0.00000, 0.58323, 0.54108, 0.43806, 0.50002, 0.10678],
         [3.00000, 0.00000, 0.46620, 0.31667, 0.31966, 0.43196, 0.30302],
         [3.00000, 0.00000, 0.77967, 0.27431, 0.20944, 0.24632, 0.06594]], device='cuda:0'),
 tensor([[0.00000, 0.00000, 0.56443, 0.34831, 0.19222, 0.21478, 0.61262],
         [0.00000, 0.00000, 0.36995, 0.35142, 0.32335, 0.43059, 0.11950],
         [0.00000, 0.00000, 0.57804, 0.22893, 0.33040, 0.45786, 0.10753],
         [0.00000, 0.00000, 0.61376, 0.37359, 0.32855, 0.40408, 0.05609],
         [1.00000, 0.00000, 0.67428, 0.71190, 0.44346, 0.57621, 0.05826],
         [1.00000, 0.00000, 0.48056, 0.82444, 0.26191, 0.35112, 0.05710],
         [1.00000, 

In [229]:
plot_images(deepcopy(img), pseudo_label, path, "images_out.jpg")
plot_images(deepcopy(img), pseudo_label1, path, "images_out1.jpg")
plot_images(deepcopy(img), pseudo_label1_aug, path, "images_out1_aug.jpg")

plot_images(deepcopy(img), labels_out, path, "images_label.jpg")


array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [114, 114, 114],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]